# Working with personal data requests
In this notebook, I'll show how I imported data from Netflix and from Hulu. Both are streaming video services, but they provide data in very different ways:
* Netflix gave  me data as a series of csv files
* Hulu gave me a single PDF

In [1]:
import pandas as pd
import numpy as np

### First the easy one: Netflix

In [2]:
netflix = pd.read_csv("ViewingActivity.csv")

In [3]:
netflix.head()

Profile Name           Start Time  Duration  \
0       Jordan  2023-02-25 04:22:46  00:14:01   
1       Jordan  2023-02-25 03:48:38  00:34:07   
2       Jordan  2023-02-24 07:11:22  00:22:47   
3       Jordan  2023-02-20 04:09:57  00:02:38   
4       Jordan  2023-02-20 03:39:06  00:30:51   

                        Attributes  \
0  Autoplayed: user action: None;    
1                              NaN   
2                              NaN   
3                              NaN   
4                              NaN   

                                               Title Supplemental Video Type  \
0  Perfect Match: Season 1: Love Is Savage (Episo...                     NaN   
1   Perfect Match: Season 1: Blind-Sided (Episode 5)                     NaN   
2   Perfect Match: Season 1: Blind-Sided (Episode 5)                     NaN   
3  Love Is Blind: Season 3: After the Altar: A Se...                     NaN   
4  Love Is Blind: Season 3: After the Altar: The ...                     NaN   

                            Device Type  Bookmark  Latest Bookmark  \
0  Netflix Firefox MAC (Cadmium) HTML 5  00:14:07         00:14:07   
1  Netflix Firefox MAC (Cadmium) HTML 5  00:56:50         00:56:50   
2           DefaultWidevineAndroidPhone  00:22:47  Not latest view   
3           DefaultWidevineAndroidPhone  00:02:43         00:02:43   
4           DefaultWidevineAndroidPhone  00:30:55         00:30:55   

              Country  
0  US (United States)  
1  US (United States)  
2  US (United States)  
3  US (United States)  
4  US (United States)

The timestamp data we want is in the "Start Time" column. So all we have to do is extract that and then save it as a CSV, whoohoo!

In [4]:
# pandas has a built in method for exporting to CSVs, to_csv. 
# it's basically the inverse of read_csv
netflix["Start Time"].to_csv("netflix_timestamps.csv")

### Now the not so easy one, Hulu

**Hulu** gave me my personal watch history data in the form of a PDF, so I had to figure out how to get it in a workable format (i.e. a csv). 

I used [Tabula](https://pypi.org/project/tabula-py/) (a Java tool, which runs in a Python wrapper) to extract the text from the PDF into a table format. _(Note: To use it, I had to install/update Java, and then install the Tabula package! A bit annoying, but not too bad...)_

Once I did that, though, it was fairly easy to run.

If you are having issues getting Tabula installed on your machine:
* make sure you try following the installation instructions [here](https://pypi.org/project/tabula-py/)
* you can also try running it in a Google Colab notebook, example [here](https://colab.research.google.com/github/chezou/tabula-py/blob/master/examples/tabula_example.ipynb)

In [5]:
import tabula

In [ ]:
# Read pdf into list of DataFrames
df = tabula.read_pdf("hulu-redacted.pdf", pages='all')

Got stderr: Oct 30, 2023 1:37:05 PM org.apache.pdfbox.contentstream.operator.graphics.ClosePath process
Oct 30, 2023 1:37:05 PM org.apache.pdfbox.contentstream.operator.graphics.ClosePath process
Oct 30, 2023 1:37:05 PM org.apache.pdfbox.contentstream.operator.graphics.ClosePath process
Oct 30, 2023 1:37:05 PM org.apache.pdfbox.contentstream.operator.graphics.ClosePath process
Oct 30, 2023 1:37:05 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
Oct 30, 2023 1:37:05 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Oct 30, 2023 1:37:07 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Oct 30, 2023 1:37:07 PM org.apache.pdfbox.contentstream.operator.graphics.ClosePath process
Oct 30, 2023 1:37:07 PM org.apache.pdfbox.contentstream.operator.graphics.ClosePath process
Oct 30, 2023 1:37:07 PM org.apache.pdfbox.contentstream.operator.graphics.ClosePath process
Oct 30, 2023 1:37:07 PM org.apache.pdfbox.contentstream.operator.graphics.ClosePat

Let's examine the result to see what it looks like. Note that it's a list with DataFrames inside it -- one for each page of the pdf.

In [7]:
# Examine the data type and length
print(type(df))
print(len(df))

<class 'list'>
15


In [8]:
# Examine an individual element
df[0]

Settings And Preferences
0                                                NaN
1                                           Profiles
2                                                NaN
3  We have no information about you or your accou...

Note how my watch history, which only goes back to March 1, 2022 (maybe Hulu only gives a year of data by default? or deletes older data?), is on pages 7 through 9. So the data frames I need are df[6], df[7], and df[8].

In [9]:
# Let's look at each of them
#df[6]
#df[7]
df[8]

Episode Name           Series Name  \
0                                              Week 9      The Bachelorette   
1                       What I Wouldn't Give for Love          Good Trouble   
2                                    The Men Tell All      The Bachelorette   
3                                              Week 7      The Bachelorette   
4                                              Week 6      The Bachelorette   
5                                              Week 4      The Bachelorette   
6                                           Episode 2      Love Island (UK)   
7                                           Episode 1      Love Island (UK)   
8                          That's Me in the Spotlight          Good Trouble   
9                                           Episode 1  Bachelor in Paradise   
10                                I Don't Belong Here          Good Trouble   
11                                 Take These Chances          Good Trouble   
12  Something Unpredictable, but in\rthe End It's ...          Good Trouble   
13              So This is What the Truth Feels\rLike          Good Trouble   
14                           It's Lonely Out in Space          Good Trouble   
15                                  Meet the New Boss          Good Trouble   
16                           Kiss Me and Smile for Me          Good Trouble   
17                          Turn and Face the Strange          Good Trouble   
18                                     Week 10, Pt. 2          The Bachelor   
19                                            Week 10          The Bachelor   
20                                             Week 9          The Bachelor   
21                                 The Women Tell All          The Bachelor   
22                                             Week 8          The Bachelor   

    Season        Last Played At  
0       19  07 Sep 2022 04:27:00  
1        4  06 Sep 2022 01:28:02  
2       19  01 Sep 2022 02:58:45  
3       19  27 Aug 2022 09:52:58  
4       19  18 Aug 2022 00:26:12  
5       19  10 Aug 2022 00:23:40  
6        8  01 Jul 2022 15:41:30  
7        8  01 Jul 2022 14:55:32  
8        4  30 May 2022 19:23:05  
9        3  30 May 2022 19:19:31  
10       4  18 May 2022 12:49:13  
11       4  24 Apr 2022 13:07:04  
12       4  16 Apr 2022 13:02:47  
13       4  08 Apr 2022 00:16:40  
14       4  02 Apr 2022 15:43:09  
15       4  25 Mar 2022 23:09:58  
16       4  17 Mar 2022 20:52:36  
17       4  17 Mar 2022 19:58:29  
18      26  17 Mar 2022 11:24:15  
19      26  15 Mar 2022 14:05:17  
20      26  10 Mar 2022 02:10:03  
21      26  09 Mar 2022 23:49:18  
22      26                   NaN

...df[7] and df[8] are fine, but we need to do a bit of cleaning up on df[6]

In [10]:
# Need to strip top row, set row 1 as the headers

new_header = df[6].iloc[1] #grab the first row for the header
viewing_hist_1 = df[6][2:] #take the data less the header row
viewing_hist_1.columns = new_header #set the header row as the df header

# Now need to concatenate the three pages
viewing_hist = pd.concat([viewing_hist_1,df[7],df[8]])

In [14]:
viewing_hist

Episode Name              Series Name Season        Last Played At
2               Week 5             The Bachelor     27  22 Feb 2023 07:03:31
3               Week 4             The Bachelor     27  15 Feb 2023 07:25:59
4                Pilot  A Million Little Things      1  08 Feb 2023 07:55:26
5               Week 3             The Bachelor     27  08 Feb 2023 07:55:26
6               Week 2             The Bachelor     27  01 Feb 2023 15:32:46
..                 ...                      ...    ...                   ...
18      Week 10, Pt. 2             The Bachelor     26  17 Mar 2022 11:24:15
19             Week 10             The Bachelor     26  15 Mar 2022 14:05:17
20              Week 9             The Bachelor     26  10 Mar 2022 02:10:03
21  The Women Tell All             The Bachelor     26  09 Mar 2022 23:49:18
22              Week 8             The Bachelor     26                   NaN

[79 rows x 4 columns]

We have some NaNs... let's drop those.

In [15]:
viewing_hist = viewing_hist.dropna()

In [16]:
# And finally, export to a csv file
viewing_hist["Last Played At"].to_csv("hulu_timestamps.csv")

### What if you have _other_ types of data?

A summary of pandas input/output (I/O) tools is [here](https://pandas.pydata.org/docs/user_guide/io.html). Here are some things we can directly import:
* CSVs
* JSON
* text (rtf, txt)
* HTML
* MS Excel
* Pickle (Python file format)
* Parquet
* And many others...

With HTML, you might want to parse it using Beautiful Soup. With PDFs, I suggest using Tabula.

### Your task

Import your own personal data file (or better yet, several of them!). Extract the timestamp data. Export it as a CSV file and save it.
